# Daten kombinieren

**Inhalt:** Verschiedene Quellen kombinieren, optisch und tabellarisch auswerten

**Nötige Skills:** Daten explorieren, filtern, klassifizieren

**Lernziele:**
- Besser verstehen, wie Merge funktioniert
- Daten kombinieren, Hypothesen testen
- Einführung in Geopandas

# Das Beispiel

Der Immobilienboom hat die Schweiz über die letzten Jahre stark geprägt.

Dazu haben wir Daten aus verschiedenen Quellen, die wir kombinieren möchten:
- Bevölkerungsdaten aus Schweizer Gemeinden (BFS)
- Wohnungsbau in Schweizer Gemeinden (BFS)
- Leerwohnungsziffer in Schweizer Gemeinden (BFS)

Anhand dieser Daten wollen wir herausfinden:
- Welche Gemeinden sind am meisten gewachsen? Wo wurde am stärksten am Markt vorbeigebaut?

Und natürlich wollen wir unsere Resultate schön darstellen.

## Vorbereitung

Wir importieren ausnahmsweise etwas mehr Bibliotheken als sonst...

In [ ]:
import pandas as pd

In [ ]:
import geopandas as gpd

In [ ]:
from shapely.geometry import Point

In [ ]:
import matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

## Daten laden

### Einwohnerzahlen

Wir starten mit einem File, das die Einwohnerzahl der Schweizer Gemeinden beinhaltet, für alle Jahre von 1981 bis 2016.

In [ ]:
df_bev = pd.read_excel('dataprojects/immobilienboom/data/BFS Gemeinden Einwohnerzahlen.xlsx')

Check: Wie viele Gemeinden sind enthalten?

In [ ]:
df_bev.shape

In [ ]:
df_bev.head(2)

**Tipp:** Um sämtliche Spalten anzuzeigen, können wir bei Pandas die Display-Option anders einstellen:

In [ ]:
pd.set_option("display.max_columns", 50)

In [ ]:
df_bev.head(2)

### Wohnungszahlen

Wir haben drei Files für die Anzahl der Wohnungen:

- **eines für die totale Anzahl Wohnungen pro Gemeinde, von 2009 bis 2016**

In [ ]:
df_wohn_tot = pd.read_excel('dataprojects/immobilienboom/data//BFS Anzahl Wohnungen Total.xlsx')

**Check:** Wie viele Gemeinden sind im File?

In [ ]:
df_wohn_tot.shape

In [ ]:
df_wohn_tot.head(3)

- **eines für die Wohnungen in Einfamilienhäusern**

In [ ]:
df_wohn_efh = pd.read_excel('dataprojects/immobilienboom/data/BFS Anzahl Wohnungen EFH.xlsx')

In [ ]:
df_wohn_efh.head(2)

- **eines für die Anzahl der Wohnungen in Mehrfamilienhäuser**

In [ ]:
df_wohn_mfh = pd.read_excel('dataprojects/immobilienboom/data/BFS Anzahl Wohnungen MFH.xlsx')

In [ ]:
df_wohn_mfh.head(2)

### Leerwohnungsziffern

Schliesslich haben wir eine Datei, welche die aktuelle Leerwohnungsziffer für jede Gemeinde beinhaltet.

In [ ]:
df_lwz = pd.read_excel('dataprojects/immobilienboom/data/BFS Leerwohnungsziffer 2017.xlsx')

**Check:** Wie viele Gemeinden sind drin? (Hint: nicht genau gleich viele wie im anderen File...)

In [ ]:
df_lwz.shape

In [ ]:
df_lwz.head(2)

## Analyse

Wir möchten nun die drei (fünf) Tabellen gemeinsam auswerten, und zwar im Hinblick auf Fragen wie:
- Was sind Gemeinden, die einen starken Bauboom hatten und nun eine hohe Leerwohnungsziffer aufweisen?
- Haben die Leerstände etwas mit der Gemeindegrösse zu tun, gibt es in Städten höhere/tiefere Ziffern als in Dörfern?

Um Fragen dieser Art zu beantworten, müssen wir uns zwei Dinge überlegen:
1. Was sind geeignete Indikatoren, die wir konkret bilden wollen, um unsere Fragen zu beantworten?
2. Wie bringen wir die verschiedenen Indikatoren zusammen, um sie gemeinsam auszuwerten?

Wir starten mal mit der ersten Überlegung...

### Indikatoren bilden

Wir wählen zu Analysezwecken drei Indikatoren aus:
- das Bevölkerungswachstum zwischen 2009 und 2016, in Prozent
- das Wohnungswachstum (total) zwischen 2009 und 2016, in Prozent
- die Leerwohungsziffer 2017, in Prozent

Zwei der drei Indikatoren müssen wir erst noch bilden.

In [ ]:
df_bev['Bevölkerungswachstum'] = (df_bev['2016'] / df_bev['2009'] - 1) * 100

In [ ]:
df_bev.head(2)

In [ ]:
df_wohn_tot['Wohnungswachstum Total'] = (df_wohn_tot['2016'] / df_wohn_tot['2009'] - 1) * 100

In [ ]:
df_wohn_tot.head(2)

### Tabellen zusammenführen

Nun müssen wir die drei Indikatoren in einer Tablle zusammenbringen. Wir benutzen dazu die Funktion `merge()`:

**1. zunächst schliessen wir die Wohnungszahlen mit den Bevölkerungszahlen zusammen:**

In [ ]:
df_1 = df_bev.merge(df_wohn_tot, how='inner', on='Gemeindenummer')
df_1.head(2)

Pandas hat nun anhand der Gemeindenummer (einem unique identifier) die beiden Dataframes fusioniert.

**Frage:** Wie viele Gemeinden sind im neuen Dataframe enthalten? Woher kommt diese Zahl? 

Und: Wie viele Gemeinden wären enthalten, wenn wir `how='outer'` gewählt hätten? Warum?

In [ ]:
#Antwort
df_1.shape

In [ ]:
df_bev.merge(df_wohn_tot, how='outer', on='Gemeindenummer').shape

**Quiz:** Finden Sie die fünf Gemeinden, die in der Bevölkerungszahlen-Tabelle nicht enthalten sind.

In [ ]:
#Antwort


**2. Nun nehmen wir noch die Leerwohnungsziffer dazu** 

In [ ]:
df_2 = df_1.merge(df_lwz, how='inner', on='Gemeindenummer')

In [ ]:
df_2.head(2)

**Tipp:** Es lohnt sich, bei merge-Operationen ein neues Dataframe zu erstellen, sonst gibt es schnell mal ein Chaos.

Weil wir an den meisten Infos in diese Tabelle gar nicht interessiert sind, selektieren wir etwas, auch der Übersichtlichkeit halber.

Wir wählen nur unsere drei Indikatoren sowie den aktuellen Bevölkerungsstand.

In [ ]:
df_3 = df_2[['Gemeindenummer', 'Gemeinde', '2016_x', 'Bevölkerungswachstum', 'Wohnungswachstum Total', 'Leerwohnungsziffer']].copy()
df_3.rename(columns={'2016_x': 'Bevölkerung'}, inplace=True)
df_3.head(2)

Wir sind nun bereit für eine erste Analyse - am einfachsten mit ein paar Scatterplots, da sieht man schnell mal, ob etwas zusammenhängt.

### Charts

**Frage:** Wie hängen Wohnungswachstum und Leerwohnungsziffer zusammen?

In [ ]:
df_3.plot(kind='scatter', x='Bevölkerungswachstum', y='Leerwohnungsziffer', figsize=(10,7))

Antwort: In einigen Gemeinden ist die Bevölkerung sehr stark gewachsen, dort sind die Leerstände allerdings nicht so gross. Tendenziell am meisten Leerstände gibt es in der Mitte der Verteilung. Schrumpfende Gemeinden haben meist nicht ganz so hohe Leerstände.

Wir können den Plot übrigens noch etwas aufhübschen und die Punkte nach Bevölkerungsgrösse skalieren...

In [ ]:
area = df_3['Bevölkerung'] / 1000
df_3.plot(kind='scatter', x='Bevölkerungswachstum', y='Leerwohnungsziffer', s=area, alpha=0.6, figsize=(10,7))

**Quiz:** Wie hängt das Wohnungswachstum mit dem Bevölkerungswachstum zusammen? Zeichnen Sie einen Plot.

In [ ]:
#Antwort


**Quiz:** Bilden Sie eine neue Kennzahl, die das Wohnungswachstum mit dem Bevölkerungswachstum vergleicht.

Wie verhält sich diese Kennzal zur Leerwohnungsziffer?

In [ ]:
#Antwort


Offensichtlich bringen uns diese Charts auch nicht viel weiter. Wir wissen jetzt zwar, dass die grossen Städte kein besonders hohes Wachstum verzeichneten und auch nicht sehr hohe Leerstände aufweisen, aber nicht viel mehr.

Da hilft nur ein: Geopandas, to the rescue!

## Geo Plots

Geopandas funktioniert sehr ähnlich wie Pandas - bietet aber zusätzliche Funktionalität!

Zuerst müssen wir ein paar Geodaten laden, so genannte shape files.

### Geodaten laden

Zuerst eines für die Schweizer Landesgrenzen...

In [ ]:
gdf_ch = gpd.read_file('dataprojects/immobilienboom/shp/g1l17.shp')
gdf_ch.head(2)

... dann eines für die Gemeinden...

In [ ]:
gdf_gemeinden = gpd.read_file('dataprojects/immobilienboom/shp/g1g17.shp')
gdf_gemeinden.head(2)

... und noch eines für die Seen!

In [ ]:
gdf_lakes = gpd.read_file('dataprojects/immobilienboom/shp/g1s17.shp')
gdf_lakes.head(2)

Das coole an Geopandas ist: Man kann diese Shapefiles mit wenigen Zeilen Code zeichnen lassen.

In [ ]:
ax = gdf_ch.plot(edgecolor='grey', color='white', alpha=1, linewidth=2, figsize=(20,13))
gdf_lakes.plot(edgecolor='grey', facecolor='lightblue', alpha=1, linewidth=1, ax=ax)
gdf_gemeinden.plot(edgecolor='grey', color='white', alpha=1, linewidth=0.2, ax=ax)
ax.axis('off')

Das hier brauchen wir später noch:

In [ ]:
title_font = {
    'family': 'sans-serif',
    'color':  'black',
    'weight': 'bold',
    'size': 20
}
point_names = {
    'family': 'sans-serif',
    'color':  'black',
    'weight': 'normal',
    'size': 10
}

### Geodaten mit den restlichen Infos verbinden

Damit wir auf der Karte nicht nur die Gemeinden einzeichnen können, sondern auch die restlichen Infos (zB die Leerwohnungsziffer), müssen wir erneut zwei Tabellen verschmelzen. Merke: In der Geodaten-Tabelle heisst das dazugehörige Index-Feld für die Gemeinenummer nicht "Gemeindenummer", sondern "GMDNR".

In [ ]:
gdf_gemeinden_m = gdf_gemeinden.merge(df_3, how="inner", left_on="GMDNR", right_on="Gemeindenummer")

In [ ]:
gdf_gemeinden_m.head(2)

### Choropleth Maps

Machen wir mal einen ersten Plot, um zu schauen, welche Gemeinden einwohnermässig am meisten gewachsen sind.

In [ ]:
#Selber Code wie oben, für die Landesgrenzen und die Seen
ax = gdf_ch.plot(edgecolor='grey', color='white', alpha=1, linewidth=2, figsize=(20,13))
gdf_lakes.plot(edgecolor='grey', facecolor='lightblue', alpha=1, linewidth=1, ax=ax)

#Code für die Gemeinden, eingefärbt nach der Spalte "Bevölkerungswachstum"
gdf_gemeinden_m.plot(edgecolor='grey', column='Bevölkerungswachstum', cmap='YlOrRd', scheme='quantiles', alpha=0.8, linewidth=0.5, ax=ax, legend=True)

my_title = "Bevölkerungswachstum in den Gemeinden zwischen 2009 und 2016"
plt.text(480000, 312000, my_title, fontdict=title_font)

ax.axis('off')

### Dot Maps

Ich persönlich mag choropleth maps nicht so - viel lieber dot maps.

Dazu müssen wir kurz die Geometrie unseres Geodataframes ändern (was das genau ist, wird später im Kurs noch behandelt)

In [ ]:
gdf_gemeinden_m['point_geometry'] = gdf_gemeinden_m.apply(lambda row: Point(row.X_CNTR, row.Y_CNTR), axis=1)

In [ ]:
gdf_gemeinden_m = gdf_gemeinden_m.set_geometry('point_geometry')

Plus, wir kreieren uns noch eine zusätzliche Spalte mit dem marker size:

In [ ]:
#play around with the marker size
gdf_gemeinden_m['markersize'] = (gdf_gemeinden_m['Bevölkerung'] ** 0.5) / 1.5

Jetzt sind wir ready für diesen Plot-Typ

In [ ]:
#Selber Code wie oben, für die Landesgrenzen und die Seen
ax = gdf_ch.plot(edgecolor='grey', color='#EEEEEE', alpha=1, linewidth=2, figsize=(20,13))
gdf_lakes.plot(edgecolor='grey', facecolor='lightblue', alpha=1, linewidth=1, ax=ax)

#Wir haben die Gemetrie zuvor auf einen Punkt gesetzt - darum kommt das jetzt etwas anders raus
gdf_gemeinden_m.plot(markersize=gdf_gemeinden_m['markersize'], column='Bevölkerungswachstum', cmap='YlOrRd', scheme='quantiles', alpha=0.7, linewidth=0.5, legend=True, ax=ax)

my_title = "Bevölkerungswachstum in den Gemeinden zwischen 2009 und 2016"
plt.text(480000, 312000, my_title, fontdict=title_font)

ax.axis('off')

Das schöne ist: Man sieht jetzt, wo die Gemeinden wirklich sind, und vor allem auch: wie gross sie sind.

Es zeigt sich: Stark gewachsen ist die Bevölkerung vor allem um den Genfersee sowie im Raum Zug-Zürich.

Machen wir gleich noch einen Plot - mit den **Leerstandsziffern**

In [ ]:
#Selber Code wie oben, für die Landesgrenzen und die Seen
ax = gdf_ch.plot(edgecolor='grey', color='#EEEEEE', alpha=1, linewidth=2, figsize=(20,13))
gdf_lakes.plot(edgecolor='grey', facecolor='lightblue', alpha=1, linewidth=1, ax=ax)

#Die Gemeinden, eingefärbt anhand der Leerwohnungsziffer
gdf_gemeinden_m.plot(markersize=gdf_gemeinden_m['markersize'], column='Leerwohnungsziffer', cmap='YlOrRd', scheme='quantiles', alpha=0.8, linewidth=0.5, legend=True, ax=ax)

my_title = "Leerwohnungsziffer 2017"
plt.text(480000, 312000, my_title, fontdict=title_font)

ax.axis('off')

Man sieht nun sehr rasch, wo die Problemzonen sind: im Mittelland, im Wallis, um den Bodensee.

Die Frage ist: Können wir die "schlimmsten" Gemeinden irgendwie identifizieren?

## Gemeinden filtern

Wir gehen jetzt einfach mal explorativ vor - und selektieren eine Reihe von Gemeinden anhand von bestimmten Kriterien, die wir als problematisch oder einfach nur als interessant erachten im Kontext des Immobilien-Baubooms der letzten Jahre.

Zum Beispiel:
- hohes Wohnungswachstum (>20%)
- hohe Leerstandsziffer (>4%)
- minimale Grösse (>500 Einwohner)

Was sind das für Gemeinden?

In [ ]:
gdf_gemeinden_1 = gdf_gemeinden_m[(gdf_gemeinden_m['Leerwohnungsziffer'] >= 4) & (gdf_gemeinden_m['Wohnungswachstum Total'] > 20) & (gdf_gemeinden_m['Bevölkerung'] > 500)]


In [ ]:
gdf_gemeinden_1.sort_values('Wohnungswachstum Total', ascending=False)[['Gemeinde', 'Bevölkerung', 'Wohnungswachstum Total', 'Leerwohnungsziffer']]


Und wo sind sie?

In [ ]:
#Selber Code wie oben, für die Landesgrenzen und die Seen
ax = gdf_ch.plot(edgecolor='grey', color='#EEEEEE', alpha=1, linewidth=2, figsize=(20,13))
gdf_lakes.plot(edgecolor='grey', facecolor='lightblue', alpha=1, linewidth=1, ax=ax)

#Die Gemeinden, eingefärbt anhand der Leerwohnungsziffer (Achtung, neue, angepasste Farbskala!)
gdf_gemeinden_1.plot(markersize=gdf_gemeinden_1['markersize'], column='Leerwohnungsziffer', cmap='YlOrRd', scheme='quantiles', alpha=0.8, linewidth=0.5, legend=True, ax=ax)

#Wir schreiben die Punkte noch an :-)
for i in gdf_gemeinden_1.index:
    y = gdf_gemeinden_1.loc[i, 'Y_CNTR']
    x = gdf_gemeinden_1.loc[i, 'X_MAX']
    gemeindename = gdf_gemeinden_1.loc[i, 'Gemeinde']
    plt.text(x, y, gemeindename, fontdict=point_names)

my_title = "Gemeinden mit hohen Leerständen und grossem Bauboom"
plt.text(480000, 312000, my_title, fontdict=title_font)

ax.axis('off')

Falls wir mal eine Reportage aus einer Gemeinde machen wollen, in der viel gebaut wurde - am Markt vorbei - wären das hier die Kandidaten!

## Und nun, zum selber machen...

Welches sind Gemeinden, die man sich näher anschauen sollte?
1. Wählen Wie Indikatoren aus, der Sie interessiert (zB: Einfamilienhäuser, Leerstände)
1. Verbinden Sie die Datensets
1. Geodaten und restliche Daten verbinden
1. Filtern Sie eine Reihe von Gemeinden heraus, die speziell auffallen
1. Plotten Sie diese Gemeinden auf einer Karte

Keine Angst: Geopandas-Funktionen kommen erst im letzten Schritt rein.

### 1. Indikatoren wählen

Gehen Sie nochmals ganz nach oben zurück, zum Teil "Analyse".

Und berechnen Sie eine Reihe von Indikatoren, die Sie näher untersuchen wollen, aus den Tabellen "df_bev", "df_wohn_tot", "df_wohn_EFH", "df_wohn_MFH", "df_lwz". (Oder wenn Sie ganz verwegen sind: Holen Sie sich vom www.bfs.admin.ch irgendeine andere Statistik auf Gemeindeebene, welche die Gemeindenummer beinhaltet!)

### 2. Tabellen zusammenführen

Benutzen Sie die Funktion `merge()`, um ihre Zahlen in einer einzigen Tabelle zu verschmelzen!

### 3. Geodaten mit Tabellen verbinden
Holen Sie sich nochmals das originale Geodataframe "gdf_gemeinden". Führen Sie dieses zusammen mit ihrer Tabelle.

### 4. Gemeinden filtern
Spielen Sie mit den Parametern so lange, bis Sie eine Liste von ungefähr 30 Gemeinden erhalten, die alle ihre Kriterien erfüllen.

### 5. Dot Map der gefilterten Gemeinden
Die hohe Kunst des Programmierens ist: Code kopieren, Code anpassen. Nicht vergessen: Punkt-Geometrie im Geodataframe nochmals setzen!